In [1]:
from matplotlib import pyplot as plt
import numpy as np

data = np.load('data/lhq_256/24_classes_rbg.npy')


In [10]:
np.unique(data[:760].reshape((-1, 3)), axis=0).shape

(25, 3)

In [ ]:
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
from skimage import io, transform, color
import requests
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from typing import List
from dataset import parse_csv
from tqdm import tqdm
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")

In [ ]:
"""
150 classes
we need:
1. sky (2)
2. tree (4)
3. grass (9)
4. earth;ground (13)
5. mountain;mount (16)
6. plant;flora;plant;life (17)
7. sea (27)
8. 
"""

def image_to_mask(images, model, feature_extractor) -> List[np.ndarray]:
    inputs = feature_extractor(images=images, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    data: np.ndarray = np.array(logits.detach().numpy()).transpose((0, 2, 3, 1))
    image_mask = np.argmax(data, axis=-1, keepdims=True)
    return image_mask.astype(np.uint8)

def read_images(paths):
    return [io.imread(p) for p in paths]

classes = parse_csv()

files = glob('./data/lhq_256/images/*.png')
batch = 64
for i in tqdm(range(0, len(files), batch)):
    paths = files[i:i+batch]
    images = [io.imread(p) for p in paths]
    masks = image_to_mask(images, model, feature_extractor)
    for k in range(len(masks)):
        name = paths[k].replace('images', 'annotations_128')
        io.imsave(name, masks[k], check_contrast=False)

In [ ]:
import multiprocessing
import itertools
from typing import List
import numpy as np
import pandas as pd
from glob import glob

def parse_csv(path: str = 'data/ADE20K/included_classes.csv', sep: str = ';'):
    csv = pd.read_csv(path, sep=sep)
    classes = {}
    for index, (name, inc) in enumerate(zip(csv['Name'], csv['Include'])):
        if inc:
            classes[name] = index
    return classes    



if __name__ == '__main__':
    classes = parse_csv()


In [ ]:
import keras.backend as K
d = K.variable([1, 1, 10])
K.set_value(d, [1, 1, 4])
d